### Cписував тут -> https://github.com/konopl1tsky1/geo_data_analisis/tree/main/lab_07

In [1]:
import geopandas
import rasterio
from rasterio.mask import mask
import glob
import os

In [2]:
field_polygons = geopandas.read_file("./field_polygons.geojson")
field_polygons['Name'] = field_polygons['Name'] + '_' + field_polygons.index.astype(str)
field_polygons  

,id,Name,geometry
0,1,Pin4_0,"MULTIPOLYGON (((30.17411 49.99641, 30.17705 50..."
1,2,Pin17_1,"MULTIPOLYGON (((30.24050 49.98769, 30.24404 49..."
2,3,Shev7-8-9_2,"MULTIPOLYGON (((30.32204 50.00902, 30.33148 50..."
3,4,Shev10_3,"MULTIPOLYGON (((30.35295 50.01389, 30.35007 50..."
4,5,Tr5_4,"MULTIPOLYGON (((30.33304 50.02297, 30.34076 50..."
5,6,Shev6_5,"MULTIPOLYGON (((30.31524 50.02392, 30.31583 50..."
6,7,Tr4_6,"MULTIPOLYGON (((30.34083 50.03253, 30.34986 50..."
7,8,Tr3_7,"MULTIPOLYGON (((30.36117 50.04025, 30.36097 50..."
8,9,Tr2_8,"MULTIPOLYGON (((30.35835 50.04439, 30.35962 50..."
9,10,Tr10_9,"MULTIPOLYGON (((30.33419 50.02495, 30.32681 50..."


In [3]:
tif_files = glob.glob('./spatial_data/**/*.tif', recursive=True)
print(len(tif_files))

68


In [4]:
if not os.path.exists('./resampled'):
    os.mkdir('./resampled')

for name in field_polygons['Name']:
    if ('/' in name):
        name = name.replace('/', '_')
    dir_name = os.path.join('./resampled', name)
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

In [5]:
list_spatial_path = './spatial_data'
list_spatial_data = [f for f in os.listdir(list_spatial_path) if os.path.isdir(os.path.join(list_spatial_path, f))]

for index, row in field_polygons.iterrows():
    geometry = row['geometry']
    name = row['Name']

    for spatial_data_folder in list_spatial_data:
        spatial_data_file_name = os.listdir(os.path.join(list_spatial_path, spatial_data_folder))[0]
        spatial_data_file_path = os.path.join(list_spatial_path, os.path.join(spatial_data_folder, spatial_data_file_name))

        spatial_data_file = rasterio.open(spatial_data_file_path)
        cropped_spatial_data, cropped_spatial_transform = mask(spatial_data_file, shapes=[geometry], crop=True)

        out_meta = spatial_data_file.meta.copy()
        out_meta.update({"driver": "GTiff",
            "height": cropped_spatial_data.shape[1],
            "width": cropped_spatial_data.shape[2],
            "transform": cropped_spatial_transform})
        
        if ('/' in name):
            name = name.replace('/', '_')

        out_folder_path = os.path.join(os.path.join('./resampled', name), spatial_data_folder)
        out_file_path = os.path.join(out_folder_path, spatial_data_file_name)
        if not os.path.exists(out_folder_path):
            os.mkdir(out_folder_path)

        with rasterio.open(out_file_path, 'w', **out_meta) as f:
            f.write(cropped_spatial_data)